In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import min, max, avg, col, udf

In [0]:
spark = SparkSession.builder.appName('Office Data Project').getOrCreate()

In [0]:
df = spark.read.options(header=True, inferSchema=True).csv('/FileStore/tables/OfficeDataProject.csv')

In [0]:
df.show()

+-----------+-------------------+----------+-----+------+---+-----+
|employee_id|      employee_name|department|state|salary|age|bonus|
+-----------+-------------------+----------+-----+------+---+-----+
|       1000|          Nitz Leif| Marketing|   CA|  6131| 26|  543|
|       1001|    Melissia Dedman|   Finance|   AK|  4027| 43| 1290|
|       1002|  Rudolph Barringer|        HR|   LA|  3122| 43| 1445|
|       1003|        Tamra Amber|  Accounts|   AK|  5717| 47| 1291|
|       1004|        Mullan Nitz|Purchasing|   CA|  5685| 34| 1394|
|       1005|      Zollner Karie|  Accounts|   CA|  2843| 27| 1078|
|       1006|Kaczorowski Zollner|     Sales|   CA|  7201| 21| 1834|
|       1007|      Nakano Locust| Marketing|   LA|  3444| 23| 1823|
|       1008|  Recalde Kensinger|  Accounts|   LA|  3704| 48| 1330|
|       1009|        Imai Hallie|  Accounts|   AK|  5061| 38| 1557|
|       1010|    Debroah Gallman|  Accounts|   NY|  9308| 35|  817|
|       1011|   Barringer Escoto|Purchasing|   W

In [0]:
print(f'Total employees = %d' %df.select('employee_id').distinct().count())

Total employees = 1000


In [0]:
print(f'Total Depts = %d' %df.select('department').distinct().count())

Total Depts = 6


In [0]:
df.select('department').distinct().show()

+----------+
|department|
+----------+
|     Sales|
|        HR|
|   Finance|
|Purchasing|
| Marketing|
|  Accounts|
+----------+



In [0]:
df.groupBy('department').count().show()

+----------+-----+
|department|count|
+----------+-----+
|     Sales|  169|
|        HR|  171|
|   Finance|  162|
|Purchasing|  166|
| Marketing|  170|
|  Accounts|  162|
+----------+-----+



In [0]:
df.groupBy('state').count().show()

+-----+-----+
|state|count|
+-----+-----+
|   LA|  205|
|   CA|  205|
|   WA|  208|
|   NY|  173|
|   AK|  209|
+-----+-----+



In [0]:
df.groupBy('state','department').count().show()

+-----+----------+-----+
|state|department|count|
+-----+----------+-----+
|   CA|     Sales|   42|
|   CA| Marketing|   33|
|   NY|  Accounts|   34|
|   NY|     Sales|   27|
|   CA|   Finance|   35|
|   CA|  Accounts|   35|
|   CA|Purchasing|   32|
|   WA|        HR|   47|
|   AK|Purchasing|   30|
|   WA|  Accounts|   27|
|   WA|Purchasing|   38|
|   AK|     Sales|   38|
|   AK|  Accounts|   37|
|   WA| Marketing|   39|
|   LA|        HR|   41|
|   LA|     Sales|   35|
|   AK|        HR|   25|
|   LA|   Finance|   29|
|   AK|   Finance|   37|
|   LA|Purchasing|   45|
+-----+----------+-----+
only showing top 20 rows



In [0]:
df.groupBy('department').agg(min('salary').alias('min salary'), max('salary').alias('max salary')).sort(col('max salary').asc(),col('min salary').asc()) .show()

+----------+----------+----------+
|department|min salary|max salary|
+----------+----------+----------+
|  Accounts|      1007|      9890|
|   Finance|      1006|      9899|
| Marketing|      1031|      9974|
|        HR|      1013|      9982|
|     Sales|      1103|      9982|
|Purchasing|      1105|      9985|
+----------+----------+----------+



In [0]:
avg_bonus = df.filter(col('state') == 'NY').groupBy(col('state')).agg(avg(col('bonus')).alias('avg')).select('avg').collect()[0]['avg']
df.filter((col('state') == 'NY') & (col('department') == 'Finance') & (col('bonus') > avg_bonus)).show()

+-----------+--------------------+----------+-----+------+---+-----+
|employee_id|       employee_name|department|state|salary|age|bonus|
+-----------+--------------------+----------+-----+------+---+-----+
|       1035|       Vivan Sifford|   Finance|   NY|  1129| 35| 1261|
|       1073|      Herder Gallman|   Finance|   NY|  1988| 31| 1402|
|       1082|          Nena Rocha|   Finance|   NY|  3417| 25| 1647|
|       1087|       Leif Lemaster|   Finance|   NY|  8642| 45| 1782|
|       1100|Ellingsworth Meli...|   Finance|   NY|  7845| 32| 1358|
|       1127|        Escoto Gilma|   Finance|   NY|  3426| 41| 1285|
|       1161|     Georgeanna Laub|   Finance|   NY|  2469| 26| 1679|
|       1175|     Durio Tenenbaum|   Finance|   NY|  2253| 42| 1684|
|       1180|       Juliana Grigg|   Finance|   NY|  8178| 42| 1617|
|       1215|        Tiffani Benz|   Finance|   NY|  1665| 41| 1969|
|       1220|          Nitz Ilana|   Finance|   NY|  2443| 50| 1342|
|       1342|   Phylicia Antonina|

In [0]:
def increament(age,salary):
    return salary + 500 if age > 45 else salary

In [0]:
inc_udf = udf(lambda age, sal: increament(age,sal))

In [0]:
df.withColumn('new_sal', inc_udf(col('age'), col('salary'))).show()

+-----------+-------------------+----------+-----+------+---+-----+-------+
|employee_id|      employee_name|department|state|salary|age|bonus|new_sal|
+-----------+-------------------+----------+-----+------+---+-----+-------+
|       1000|          Nitz Leif| Marketing|   CA|  6131| 26|  543|   6131|
|       1001|    Melissia Dedman|   Finance|   AK|  4027| 43| 1290|   4027|
|       1002|  Rudolph Barringer|        HR|   LA|  3122| 43| 1445|   3122|
|       1003|        Tamra Amber|  Accounts|   AK|  5717| 47| 1291|   6217|
|       1004|        Mullan Nitz|Purchasing|   CA|  5685| 34| 1394|   5685|
|       1005|      Zollner Karie|  Accounts|   CA|  2843| 27| 1078|   2843|
|       1006|Kaczorowski Zollner|     Sales|   CA|  7201| 21| 1834|   7201|
|       1007|      Nakano Locust| Marketing|   LA|  3444| 23| 1823|   3444|
|       1008|  Recalde Kensinger|  Accounts|   LA|  3704| 48| 1330|   4204|
|       1009|        Imai Hallie|  Accounts|   AK|  5061| 38| 1557|   5061|
|       1010

In [0]:
inc_udf1 = udf(lambda age, sal: sal + 500 if age > 45 else sal)

In [0]:
df.withColumn('new_sal', inc_udf1(col('age'), col('salary'))).show()

+-----------+-------------------+----------+-----+------+---+-----+-------+
|employee_id|      employee_name|department|state|salary|age|bonus|new_sal|
+-----------+-------------------+----------+-----+------+---+-----+-------+
|       1000|          Nitz Leif| Marketing|   CA|  6131| 26|  543|   6131|
|       1001|    Melissia Dedman|   Finance|   AK|  4027| 43| 1290|   4027|
|       1002|  Rudolph Barringer|        HR|   LA|  3122| 43| 1445|   3122|
|       1003|        Tamra Amber|  Accounts|   AK|  5717| 47| 1291|   6217|
|       1004|        Mullan Nitz|Purchasing|   CA|  5685| 34| 1394|   5685|
|       1005|      Zollner Karie|  Accounts|   CA|  2843| 27| 1078|   2843|
|       1006|Kaczorowski Zollner|     Sales|   CA|  7201| 21| 1834|   7201|
|       1007|      Nakano Locust| Marketing|   LA|  3444| 23| 1823|   3444|
|       1008|  Recalde Kensinger|  Accounts|   LA|  3704| 48| 1330|   4204|
|       1009|        Imai Hallie|  Accounts|   AK|  5061| 38| 1557|   5061|
|       1010

In [0]:
df.filter(col('age') > 45).write.options(header=True).mode('overwrite').csv('/FileStore/tables/ProjectDF/output')